In [3]:
import requests
import pandas as pd

url = 'https://api.apify.com/v2/datasets/tG0NVmk7nrsTOq39S/items'
response = requests.get(url)
data = response.json()

spouštět až odsud dolů (v případě chyb, testování...), ať pořád nevoláme api.

In [4]:
df = pd.DataFrame(data)

df.head(2)

,externalId,branchCode,name,distanceFromOrigin,address,coordinates,flags,openingHours,googleMapUrl,canBeSelected
0,I_12714,z-box-ceska-kubice-ceska-kubice-82,Z-BOX,100251.85,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'isNew': False, 'isClaimAssistant': False, 'i...","{'regular': {'monday': [{'open': '00:00', 'clo...",https://www.google.com/maps/dir/?api=1&destina...,True
1,I_12712,z-box-vseruby-u-kdyne-vseruby-40,Z-BOX,99734.36,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'isNew': False, 'isClaimAssistant': False, 'i...","{'regular': {'monday': [{'open': '00:00', 'clo...",https://www.google.com/maps/dir/?api=1&destina...,True


Kolik je v DF řádků a sloupců. Je nutno průběžně kontrolovat.

In [5]:
df.shape

(5327, 10)

Vybrat v DF sloupce, ostaní zahodit (nebudou třeba 100% jistota)

In [6]:
df = df[['name', 'address', 'coordinates', 'openingHours']]

df.columns
df.head(2)

,name,address,coordinates,openingHours
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."


1ADRESA - rozdělení sloupce address
Postup v jedn. krocích, mj. i pro zpětnou kontrolu. 
Celý postup několikrát opravuji.

1A Vytáhnout hodnotu podle klíče 'name' do sloupce 'address_text'
   Vybrat sloupce pro lepší přehlednost

In [7]:
df['address_text'] = df['address'].str.get('name')
df = df[['name', 'address_text', 'coordinates', 'openingHours']]

df.columns
df.head(2)

,name,address_text,coordinates,openingHours
0,Z-BOX,"Z-BOX Česká Kubice, Česká Kubice 82","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,"Z-BOX Všeruby u Kdyně, Všeruby 40","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."


1B Odstranit 'přesný text' 'Z-box '
   Ořezat mezery před/za

In [8]:
df['address_text'] = df['address_text'].str.replace('Z-BOX ', '', regex=False)
df['address_text'] = df['address_text'].str.strip()

df.columns
df.head(2)

,name,address_text,coordinates,openingHours
0,Z-BOX,"Česká Kubice, Česká Kubice 82","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,"Všeruby u Kdyně, Všeruby 40","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."


První čištění
První čárka zleva/city, zbývající street and No - dělím první
mezerou zprava. Chyba - city nekončí první čárkou, č.p. není vždy, za č.p. se
objevuje text v závorkách, mezery na zač a konci address text.
Druhé čištění
Snažím se jít zprava, odstaním text v závorce na konci řádku, odstraním mezery 
na zač a konci řádku. Pak chci zprava "odfiltrovat" č.p. a doplnit NaN u těch 
řádků, kde č.p. není. To je slepá cesta - nakonec si pomocí regexů umím vyfiltrovat
2 skupiny - s č.p. a bez č.p. Dál už s tím neumím pracovat, split neumím 
správně zadat.
Třetí čištění
Takže si vytvářím pomocné.csv a zkouším vymyslet jiný postup. Přemýšlím, jak napsat 
regex a vyextrahovat vše po číslo. Pak mě napadne, vyzkoušet extrahovat číslo,
regex už mám. Vracím se k postupu z druhého čištění jen nesplituju ale extrahuju.
Město,město, ulici - už jen RSPLIT zleva, 2 sloupce n=1
  
Opakovaně se vracím zpět do tohoto kroku a odstraním postupně chyby. Vizualizuju 
v csv - filtry v python pro mě nejsou dostatečné.
Tyto dodatečné úpravy možná nemusí být z kontextu jasné. 

1C Ukázka a odstranění "( TEXT)" na konci řádků

In [9]:
adresy_se_zavorkou = df[df['address_text'].str.contains(r'\(.*\)$', na=False)]

df.columns
adresy_se_zavorkou

,name,address_text,coordinates,openingHours
9,Z-BOX,"Kdyně, Nádražní 747 (Tesco)","{'latitude': 49.38909, 'longitude': 13.03639}","{'regular': {'monday': [{'open': '00:00', 'clo..."
10,Z-BOX,"Kdyně, Na Kobyle 209 (automyčka Comfort wapka)","{'latitude': 49.39257, 'longitude': 13.02652}","{'regular': {'monday': [{'open': '00:00', 'clo..."
15,Z-BOX,"Volary, Soumarská (Stabe stavebniny)","{'latitude': 48.91171, 'longitude': 13.8872}","{'regular': {'monday': [{'open': '00:00', 'clo..."
17,Z-BOX,"Vimperk, Sušická 617 (Tesco)","{'latitude': 49.05857, 'longitude': 13.77849}","{'regular': {'monday': [{'open': '00:00', 'clo..."
21,Z-BOX,"Sušice, Hrádecká 1310 (Tesco)","{'latitude': 49.24116, 'longitude': 13.52214}","{'regular': {'monday': [{'open': '00:00', 'clo..."
...,...,...,...,...
5303,Z-BOX,"Rumburk, Pražská 1677 (Tesco)","{'latitude': 50.94735, 'longitude': 14.55785}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5310,Z-BOX,"Liberec, Hejnická 225 (Pro-Doma)","{'latitude': 50.80947, 'longitude': 15.03618}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5320,Z-BOX,"Šluknov, Sladovnická 1100 (Tesco)","{'latitude': 51.00516, 'longitude': 14.454}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5325,Z-BOX,"Frýdlant, Fügnerova (automyčka Jomi)","{'latitude': 50.92302, 'longitude': 15.08627}","{'regular': {'monday': [{'open': '00:00', 'clo..."


In [10]:
df['address_text'] = df['address_text'].str.replace(r'\s*\(.*\)$', '', regex=True)
adresy_se_zavorkou = df[df['address_text'].str.contains(r'\(.*\)$', na=False)]

df.columns
adresy_se_zavorkou

,name,address_text,coordinates,openingHours


In [11]:
df.shape

(5327, 4)

1E VYFILTROVAT ŘÁDKY s Č.P. a bez Č.P.
- mega fuška, postavit správně regex. 
- teď už opravdu hodně lituji, že nepracuju v SQL
- nemůžu pracovat s RSPLIT ' ', v řádcích bez č.p. první mezera zprava může 
  být za textem
- musím rozpoznat řádek, kde číslo je a oddělit číslo zprava. Zkouším extract, 
  chci se vyhnout iloc/loc.

In [12]:
adresy_s_cislem = df[df['address_text'].str.contains(r'\s+\d+(/\d+[a-zA-Z]?)?$', na=False)]

df.columns
adresy_s_cislem

C:\Users\Sabina\AppData\Local\Temp\ipykernel_20568\1744462914.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  adresy_s_cislem = df[df['address_text'].str.contains(r'\s+\d+(/\d+[a-zA-Z]?)?$', na=False)]


,name,address_text,coordinates,openingHours
0,Z-BOX,"Česká Kubice, Česká Kubice 82","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,"Všeruby u Kdyně, Všeruby 40","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."
2,Z-BOX,"Nýrsko, Havlíčkova 474","{'latitude': 49.28962, 'longitude': 13.13596}","{'regular': {'monday': [{'open': '00:00', 'clo..."
3,Z-BOX,"Hartmanice, Hartmanice 136","{'latitude': 49.1695, 'longitude': 13.45571}","{'regular': {'monday': [{'open': '00:00', 'clo..."
4,Z-BOX,"Kašperské Hory, Dlouhá 138","{'latitude': 49.14296, 'longitude': 13.5555}","{'regular': {'monday': [{'open': '00:00', 'clo..."
...,...,...,...,...
5321,Z-BOX,"Raspenava, Fučíkova 647","{'latitude': 50.89744, 'longitude': 15.13943}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5322,Z-BOX,"Raspenava, Fučíkova 428","{'latitude': 50.90136, 'longitude': 15.13427}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5323,Z-BOX,"Frýdlant, Kodešova 1246","{'latitude': 50.92686, 'longitude': 15.07278}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5324,Z-BOX,"Frýdlant, Okružní 1260","{'latitude': 50.9223, 'longitude': 15.07966}","{'regular': {'monday': [{'open': '00:00', 'clo..."


In [13]:
adresy_bez_cisla = df[~df['address_text'].str.contains(r'\s+\d+(/\d+[a-zA-Z]?)?$', na=False)]

df.columns
adresy_bez_cisla

C:\Users\Sabina\AppData\Local\Temp\ipykernel_20568\3430309639.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  adresy_bez_cisla = df[~df['address_text'].str.contains(r'\s+\d+(/\d+[a-zA-Z]?)?$', na=False)]


,name,address_text,coordinates,openingHours
15,Z-BOX,"Volary, Soumarská","{'latitude': 48.91171, 'longitude': 13.8872}","{'regular': {'monday': [{'open': '00:00', 'clo..."
47,Z-BOX,"Lipno nad Vltavou, Lipno nad Vltavou","{'latitude': 48.63715, 'longitude': 14.2314}","{'regular': {'monday': [{'open': '00:00', 'clo..."
57,Z-BOX,"Klatovy, V Lázni","{'latitude': 49.40315, 'longitude': 13.28929}","{'regular': {'monday': [{'open': '00:00', 'clo..."
62,Z-BOX,"Lhenice, Školní","{'latitude': 48.99406, 'longitude': 14.14976}","{'regular': {'monday': [{'open': '00:00', 'clo..."
69,Z-BOX,"Plánice, Klatovská","{'latitude': 49.39317, 'longitude': 13.46463}","{'regular': {'monday': [{'open': '00:00', 'clo..."
...,...,...,...,...
5279,Z-BOX,"Liberec, Zahradní","{'latitude': 50.77608, 'longitude': 15.04419}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5288,Z-BOX,"Liberec, Tržní náměstí","{'latitude': 50.77411, 'longitude': 15.06061}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5298,Z-BOX,"Liberec, Na Zhořelci","{'latitude': 50.77113, 'longitude': 15.0883}","{'regular': {'monday': [{'open': '00:00', 'clo..."
5307,Z-BOX,"Liberec, Rychtářská","{'latitude': 50.78341, 'longitude': 15.06442}","{'regular': {'monday': [{'open': '00:00', 'clo..."


1E Číslo extrahovat do nového sloupce (zprava$)
   Vymazat číslo ze stávajícího 'address_text' 

In [14]:
df['house_no'] = df['address_text'].str.extract(r'\s(\d+(/\d+[a-zA-Z]?)?)$')[0]
df['address_text'] = df['address_text'].str.replace(r'\s(\d+(/\d+[a-zA-Z]?)?)$', '', regex = True)

df.columns
df.head(2)

,name,address_text,coordinates,openingHours,house_no
0,Z-BOX,"Česká Kubice, Česká Kubice","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",82
1,Z-BOX,"Všeruby u Kdyně, Všeruby","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",40


1E Dočistit zbývající čísla - upravit regex
zprava od konce řádku po první mezeru a za ní musí 
být 1 číslo a libovolně mnoho číslic, znaků, písmenek a spojovníků, čárek

In [15]:
df['address_text'] = df['address_text'].str.replace(r'\s\d[\dA-Za-z/,-]*$', '', regex = True)

df.columns
df.head(2)

,name,address_text,coordinates,openingHours,house_no
0,Z-BOX,"Česká Kubice, Česká Kubice","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",82
1,Z-BOX,"Všeruby u Kdyně, Všeruby","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",40


1F Rozdělit addresss_text na město a ulici(možné 0 a více čárek)

In [16]:
adresy_vice_carek = df[df['address_text'].str.contains(r'(.*,){2,}', na=False)]

df.columns
adresy_vice_carek

C:\Users\Sabina\AppData\Local\Temp\ipykernel_20568\2520350440.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  adresy_vice_carek = df[df['address_text'].str.contains(r'(.*,){2,}', na=False)]


,name,address_text,coordinates,openingHours,house_no
244,Z-BOX,"Znojmo, Přímětice, Větrná","{'latitude': 48.87825, 'longitude': 16.03653}","{'regular': {'monday': [{'open': '00:00', 'clo...",518/2
343,Z-BOX,"Břeclav, Charvátská Nová Ves, Lednická","{'latitude': 48.76663, 'longitude': 16.8464}","{'regular': {'monday': [{'open': '00:00', 'clo...",61/89
853,Z-BOX,"Písek, Vinařického 374,","{'latitude': 49.31563, 'longitude': 14.14034}","{'regular': {'monday': [{'open': '00:00', 'clo...",01
880,Z-BOX,"Brno, Dolní Heršpice, Vídeňská","{'latitude': 49.15491, 'longitude': 16.59786}","{'regular': {'monday': [{'open': '00:00', 'clo...",132/100
881,Z-BOX,"Brno, Tuřany, Sokolnická","{'latitude': 49.14483, 'longitude': 16.66543}","{'regular': {'monday': [{'open': '00:00', 'clo...",73/1
...,...,...,...,...,...
4728,Z-BOX,"Mladá Boleslav, Čejetice, Vodárenská","{'latitude': 50.41302, 'longitude': 14.88569}","{'regular': {'monday': [{'open': '00:00', 'clo...",162
4776,Z-BOX,"Most, Souš, 28. října","{'latitude': 50.52493, 'longitude': 13.6283}","{'regular': {'monday': [{'open': '00:00', 'clo...",108
5050,Z-BOX,"Teplice, Řetenice, Jateční","{'latitude': 50.64065, 'longitude': 13.80767}","{'regular': {'monday': [{'open': '00:00', 'clo...",470
5206,Z-BOX,"Děčín, Boletice nad Labem, Studniční","{'latitude': 50.74126, 'longitude': 14.19233}","{'regular': {'monday': [{'open': '00:00', 'clo...",415


In [17]:
adresy_bez_carek = df[~df['address_text'].str.contains(',', na=False)]

df.columns
adresy_bez_carek

,name,address_text,coordinates,openingHours,house_no
130,Z-BOX,Benešov nad Černou,"{'latitude': 48.72874, 'longitude': 14.62816}","{'regular': {'monday': [{'open': '00:00', 'clo...",20
187,Z-BOX,Chlum u Třeboně,"{'latitude': 48.95873, 'longitude': 14.93417}","{'regular': {'monday': [{'open': '00:00', 'clo...",499
321,Z-BOX,Vedrovice,"{'latitude': 49.02077, 'longitude': 16.37681}","{'regular': {'monday': [{'open': '00:00', 'clo...",NaN
636,Z-BOX,Nová Včelnice,"{'latitude': 49.23846, 'longitude': 15.06212}","{'regular': {'monday': [{'open': '00:00', 'clo...",523
761,Z-BOX,Kasejovice,"{'latitude': 49.46443, 'longitude': 13.73418}","{'regular': {'monday': [{'open': '00:00', 'clo...",NaN
2021,Z-BOX,Jeseník nad Odrou,"{'latitude': 49.6128, 'longitude': 17.90546}","{'regular': {'monday': [{'open': '00:00', 'clo...",256
3660,Z-BOX,Lično,"{'latitude': 50.17794, 'longitude': 16.17244}","{'regular': {'monday': [{'open': '00:00', 'clo...",NaN


In [18]:
address_text_rozdelene = df['address_text'].str.rsplit(',', n=1, expand=True)

address_text_rozdelene

,0,1
0,Česká Kubice,Česká Kubice
1,Všeruby u Kdyně,Všeruby
2,Nýrsko,Havlíčkova
3,Hartmanice,Hartmanice
4,Kašperské Hory,Dlouhá
...,...,...
5322,Raspenava,Fučíkova
5323,Frýdlant,Kodešova
5324,Frýdlant,Okružní
5325,Frýdlant,Fügnerova


In [19]:
address_text_rozdelene = df['address_text'].str.rsplit(',', n=1, expand=True)

df['street'] = address_text_rozdelene[1].str.strip()
df['city'] = address_text_rozdelene[0].str.strip()

In [20]:
df = df[['name', 'city', 'street', 'house_no', 'coordinates', 'openingHours']]

df.columns
df.head(2)

,name,city,street,house_no,coordinates,openingHours
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."


In [21]:
# Vytáhnout hodnoty 'lat' a 'long'
df['latitude'] = df['coordinates'].str.get('latitude')
df['longitude'] = df['coordinates'].str.get('longitude')

df.columns
df.head(2)

,name,city,street,house_no,coordinates,openingHours,latitude,longitude
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.36891,12.85810
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.33973,12.98162


In [22]:
# Úprava sloupce openingHours - postupně rozbalím a uvidím, co s tím.
df['regular'] = df['openingHours'].str.get('regular')

df.columns
df.head(2)

,name,city,street,house_no,coordinates,openingHours,latitude,longitude,regular
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'..."
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'..."


Tady jsem se blokla. Chtěla jsem řešit for cyklem, ale vůbec nevím, jak na to 
v Pandas

regular = {
    'monday': [{'open': '00:00', 'close': '23:59'}],
    'tuesday': [{'open': '00:00', 'close': '23:59'}],
    'wednesday': [{'open': '00:00', 'close': '23:59'}],
    'thursday': [{'open': '00:00', 'close': '23:59'}],
    'friday': [{'open': '00:00', 'close': '23:59'}],
    'saturday': [{'open': '00:00', 'close': '23:59'}],
    'sunday': [{'open': '00:00', 'close': '23:59'}],
    'exceptionDays': [
        {'from': '2025-04-18T00:00:00', 'times': [{'open': '00:00', 'close': '23:59'}]},
        {'from': '2025-04-21T00:00:00', 'times': [{'open': '00:00', 'close': '23:59'}]},
        ...
    ]
}
for den, seznam in regular.items():
    zaznam = seznam[0]
    for stav, cas in zaznam.items():
        print(den, stav, cas)

Toto se mi vypíše: to jsem si moc nepomohla.
monday open 00:00
monday close 23:59
tuesday open 00:00
tuesday close 23:59
wednesday open 00:00
wednesday close 23:59
thursday open 00:00
thursday close 23:59
friday open 00:00
friday close 23:59
saturday open 00:00
saturday close 23:59
sunday open 00:00
sunday close 23:59

Tak prostě loupu cibuli, nic jiného neumím.

In [23]:
# Úprava sloupce openingHours - postupně rozbalím a uvidím, co s tím, nic 
# lepšího neumím
df['monday'] = df['regular'].str.get('monday')
df['tuesday'] = df['regular'].str.get('tuesday')
df['wednesday'] = df['regular'].str.get('wednesday')
df['thursday'] = df['regular'].str.get('thursday')
df['friday'] = df['regular'].str.get('friday')
df['saturday'] = df['regular'].str.get('saturday')
df['sunday'] = df['regular'].str.get('sunday')

df.columns
df.head(2)

,name,city,street,house_no,coordinates,openingHours,latitude,longitude,regular,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]"
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]"


Seznam slovníků - vybrat první pozici
Nakonec rozdělit dny na open and close a je to
na moji obranu, toto jsme fakt nebrali - aneb mám začátečnické znalosti
a nebojím se je využít

In [24]:
df['monday_0'] = df['monday'].str[0]
df['tuesday_0'] = df['tuesday'].str[0]
df['wednesday_0'] = df['wednesday'].str[0]
df['thursday_0'] = df['thursday'].str[0]
df['friday_0'] = df['friday'].str[0]
df['saturday_0'] = df['saturday'].str[0]
df['sunday_0'] = df['sunday'].str[0]

df.columns
df.head(2)


,name,city,street,house_no,coordinates,openingHours,latitude,longitude,regular,monday,...,friday,saturday,sunday,monday_0,tuesday_0,wednesday_0,thursday_0,friday_0,saturday_0,sunday_0
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]",...,"[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}"
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]",...,"[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}"


Zůstávají chybné sloupce i po restartu kernelu. Umím se jich zbavit jen 
selectem v dalším kroku

In [25]:
df['monday_open'] = df['monday_0'].str.get('open')
df['monday_close'] = df['monday_0'].str.get('close')

df['tuesday_open'] = df['tuesday_0'].str.get('open')
df['tuesday_close'] = df['tuesday_0'].str.get('close')

df['wednesday_open'] = df['wednesday_0'].str.get('open')
df['wednesday_close'] = df['wednesday_0'].str.get('close')

df['thursday_open'] = df['thursday_0'].str.get('open')
df['thursday_close'] = df['thursday_0'].str.get('close')

df['friday_open'] = df['friday_0'].str.get('open')
df['friday_close'] = df['friday_0'].str.get('close')

df['saturday_open'] = df['saturday_0'].str.get('open')
df['saturday_close'] = df['saturday_0'].str.get('close')

df['sunday_open'] = df['sunday_0'].str.get('open')
df['sunday_close'] = df['sunday_0'].str.get('close')

df.columns
df.head(2)

,name,city,street,house_no,coordinates,openingHours,latitude,longitude,regular,monday,...,wednesday_open,wednesday_close,thursday_open,thursday_close,friday_open,friday_close,saturday_open,saturday_close,sunday_open,sunday_close
0,Z-BOX,Česká Kubice,Česká Kubice,82,"{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]",...,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,"{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...",49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]",...,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59


In [26]:
df = df[['name', 'city', 'street', 'house_no', 'latitude', 'longitude',	'monday_open', 'monday_close', 'tuesday_open', 'tuesday_close', 'wednesday_open', 'wednesday_close', 'thursday_open', 'thursday_close', 'friday_open', 'friday_close', 'saturday_open', 'saturday_close', 'sunday_open', 'sunday_close', 'openingHours']]

df.columns
df.head(2)

,name,city,street,house_no,latitude,longitude,monday_open,monday_close,tuesday_open,tuesday_close,...,wednesday_close,thursday_open,thursday_close,friday_open,friday_close,saturday_open,saturday_close,sunday_open,sunday_close,openingHours
0,Z-BOX,Česká Kubice,Česká Kubice,82,49.36891,12.85810,00:00,23:59,00:00,23:59,...,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,"{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,00:00,23:59,00:00,23:59,...,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,"{'regular': {'monday': [{'open': '00:00', 'clo..."


Nespouštím poslední příkaz - uložení do json souboru, protože máv df pořád 
sloupec opening hours (nedohodly jsme se, co s tím.)


In [ ]:
# import json

# df.to_json('zasilkovna_cisty_dataset.json', orient='records', force_ascii=False, indent=4)

In [ ]:
# df.to_csv('zasilkovna_cisty_dataset.json', index=False, encoding='utf-8')